# ETL Using Python

We will be ingesting data from the Microsoft AdventureWorks Sample Databases: https://learn.microsoft.com/en-us/sql/samples/adventureworks-install-configure?view=sql-server-ver16&tabs=ssms.

We can choose from either the OLTP, Data Warehouse or Lightweight '.bak' files. The Source is a database from SQL Server which is then processed using Extract-Load-Transform before reaching it's Destination as a PostgreSQL database.

So, we're using SQL Server 'AdventureWorksDW2019.bak' as the database source, before loading it into PostgreSQL using Python:

1. ELT is a fundamental workflow used in data engineering
2. The data source can be an API, a db or a flatfile.
3. The source data is extracted into a 'Staging Area', transformed into a product ready to be loaded into it's destination or target database (stored in a Data Lake or Data Warehouse).
4. We'll use Python (Pandas library) to build the ETL pipelines.

Extract: get data from a source.
Transform: structure, format or clean the data.
Load: write the data to an external target / destination.

## Sources
### Databases
RDBMS / NoSQL
### Files
csv / json / xml
### SaaS Applications
REST API's
### Application Events
Webhook

It's Extracted into the Staging Area where it's Transformed before being Loaded into a Data Warehouse to be Analyzed for BI, ML, Data Science and Analytics end products.

Another important point to note is that there are several different formats, database drivers and types of structure which need to be learned.

## PostgreSQL Set Up
### SQL Query
This query is in the pgAdmin4 (PostgreSQL db driver).

In [ ]:
"""
-- Database: AdventureWorks

-- DROP DATABASE "AdventureWorks";

CREATE DATABASE "AdventureWorks"
    WITH
    OWNER = postgres
    ENCODING = 'UTF8'
    LC_COLLATE = 'English_United States.1252'
    TABLESPACE = pg_default
    CONNECTION LIMIT = -1;
"""


This creates a database in PostgreSQL. 
* Refresh the databases file manager on the left hand side, open the 'Databases' folder to see the 'AdventureWorks' db.

### Create a New ETL User

In [ ]:
"""
-- create etl user
CREATE USER etl WITH PASSWORD 'demopass';
-- grant connect
GRANT CONNECT ON DATABASE "AdventureWorks"
"""

In [ ]:
#import needed libraries
from sqlalchemy import create_engine
import pyodbc
import pandas as pd
import os


'sqlalchemy' is the module used to interact with PostgreSQL. 'pyodbc' is the module used to query SQL Server. 'pandas' is the module used to perform the data extraction / loading. 'os' is the module used to retrieve the username and passwordwhich in this case is stored separately in the 'System -> Environment Variables -> System Variables (Lower window section)'. 

## Authentication
There are some important points to note here regarding authentication access to the database drivers.
The user's credentials could be stored in a '.sh' , '.ps' or an '.xml' file.

Grab the password from the environment variable.

In [ ]:
#get password from environment var
pwd = os.environ['PGPASS']
uid = os.environ['PGUID']
#sql db details
driver = "{SQL Server Native Client 11.0}"
server = "haq-PC"
database = "AdventureWorksDW2019;"


Define a variable to store the SQL Server driver.

In [ ]:
#extract data from sql server
def extract():
    try:
        src_conn = pyodbc.connect('DRIVER=' + driver + ';SERVER=' + server + '\SQLEXPRESS' + ';DATABASE=' + database + ';UID=' + uid + ';PWD=' + pwd)
        src_cursor = src_conn.cursor()
        # execute query
        src_cursor.execute(""" select  t.name as table_name
        from sys.tables t where t.name in ('DimProduct','DimProductSubcategory','DimProductSubcategory','DimProductCategory','DimSalesTerritory','FactInternetSales') """)
        src_tables = src_cursor.fetchall()
        for tbl in src_tables:
            #query and load save data to dataframe
            df = pd.read_sql_query(f'select * FROM {tbl[0]}', src_conn)
            load(df, tbl[0])
    except Exception as e:
        print("Data extract error: " + str(e))
    finally:
        src_conn.close()


## Transform
This is the phase where I need to check for missing values and generally clean the data by exploring the type of information present and determining the scale or units of measurement to produce a snapshot which can be used in production.

In [ ]:
#load data to postgres
def load(df, tbl):
    try:
        rows_imported = 0
        engine = create_engine(f'postgresql://{uid}:{pwd}@{server}:5432/AdventureWorks')
        print(f'importing rows {rows_imported} to {rows_imported + len(df)}... for table {tbl}')
        # save df to postgres
        df.to_sql(f'stg_{tbl}', engine, if_exists='replace', index=False)
        rows_imported += len(df)
        # add elapsed time to final print out
        print("Data imported successful")
    except Exception as e:
        print("Data load error: " + str(e))

try:
    #call extract function
    extract()
except Exception as e:
    print("Error while extracting data: " + str(e))

May need to provide a list of dependencies.